# Libraries and preparation

refs:
- https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_rag_agent_llama3_local.ipynb
- https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/agent_supervisor.ipynb?ref=blog.langchain.dev
- https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/hierarchical_agent_teams.ipynb?ref=blog.langchain.dev

MAP:REDUCE: https://langchain-ai.github.io/langgraph/how-tos/map-reduce/

In [1]:
import subprocess
import threading

#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%#=#=#                                                                          ################                       71.4%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
def start_ollama():
    t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
    t.start()

In [3]:
def pull_model(local_llm):
    !ollama pull local_llm

In [4]:
def start_model(local_llm):        
    t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", local_llm]),daemon=True)
    t2.start()

In [5]:
%%capture --no-stderr
%pip install -U scikit-learn==1.3 langchain-ai21 ragas langchain-pinecone langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python nomic[local] langchain-text-splitters

In [6]:
# Tracing and api-keys
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["TAVILY_API_KEY"] = "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"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["PINECONE_API_KEY"] = "94ef7896-1fae-44d3-b8d2-0bd6f5f664f5"
os.environ["AI21_API_KEY"] = "KlINkh5QKw3hG1b5Hr75YDO7TwGoQvzn"

Bias detection model:

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

bias_model_tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
bias_model = AutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model",from_tf=True)

2024-08-19 07:23:27.742566: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-19 07:23:27.742672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-19 07:23:27.857174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


- https://shap.readthedocs.io/en/latest/example_notebooks/text_examples/text_entailment/Textual%20Entailment%20Explanation%20Demo.html
- https://huggingface.co/facebook/bart-large-mnli

Entailment model (BART):

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

device = 0 if torch.cuda.is_available() else -1

bart_model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli",device=device)
bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
def BART_prediction(premise,hypothesis):
    #print(f"Premise: {premise}")
    #print(f"Hypo: {hypothesis}")
    input_ids = bart_tokenizer.encode(premise, hypothesis, return_tensors="pt")
    logits = bart_model(input_ids)[0]
    probs = logits.softmax(dim=1)

    max_index = torch.argmax(probs).item()

    bart_label_map = {0: "contradiction", 1: "neutral", 2: "entailment"}
    return bart_label_map[max_index]

# Tools

refs:
- https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/

In [10]:
### Search
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_text_splitters import RecursiveCharacterTextSplitter

web_search_tool = TavilySearchResults(k=2)

# Indexing

Organizing external sources for the llm. Phase of indexing and chunking of docs refs:
- https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/
- https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/
- https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/
- Nomic embeddings: https://docs.nomic.ai/atlas/capabilities/embeddings#selecting-a-device

osservazione: si possono controllare gli indici direttamente da https://app.pinecone.io/organizations/-O2Tiw_0VD7HTOASPJE5/projects/2a95c518-e514-4d39-bed8-4b12fd90ad44/indexes

osservazione sul chuncking: https://dev.to/peterabel/what-chunk-size-and-chunk-overlap-should-you-use-4338

In [11]:
from langchain_pinecone import PineconeVectorStore
from langchain_ai21 import AI21Embeddings

def create_retriever(index_name,top_k):
    vectorstore = PineconeVectorStore(
        index_name=index_name,
        embedding=AI21Embeddings(device="cuda")
    )
    return vectorstore.as_retriever(search_kwargs={"k": top_k})

def create_KBT_retrievers(aspects,top_k):
    retrievers = []
    for aspect in aspects:
        retriever = create_retriever(f"{aspect.lower()}-kbt",top_k)
        retrievers.append(retriever)
    return retrievers

# Prompt support

In [12]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_community.llms import Ollama

def create_prompt(system_prompt, input_variables, examples=None,):
    user_message = [("user", f"{{{input_var}}}") for input_var in input_variables]
    final_messages = [("system",system_prompt)]
    
    if examples!=None:
        example_message = [("user", f"{{{input_var}}}") for input_var in input_variables]
        example_message.append(("assistant", "{output}"))
        example_prompt  = ChatPromptTemplate.from_messages(example_message)
        few_shot_prompt = FewShotChatMessagePromptTemplate(
            example_prompt=example_prompt,
            examples=examples, 
        )
        final_messages.append(few_shot_prompt)
    
    for mess in user_message:
        final_messages.append(mess)
    #print(final_messages)
    return ChatPromptTemplate.from_messages(final_messages)

In [13]:
def call_model(llm: str, prompt: str, input_variables:list[str], examples=None):
    model = Ollama(model=llm, temperature=0)
    prompt_final = create_prompt(prompt, input_variables, examples)
    return prompt_final | model

In [14]:
def get_prompt(path):
    with open(path, 'r') as file:
        prompt = file.read()
    return prompt

def get_examples(path):
    with open(path, 'r') as file:
        examples = file.read()
    return eval(examples)

https://learnprompting.org/docs/reliability/debiasing

# Aspect agents

refs
- https://www.langchain.com/langgraph

In [15]:
from pprint import pprint
from typing import List, Annotated
import operator
import functools
import sklearn.metrics
import numpy as np

from langchain_core.documents import Document
from typing_extensions import TypedDict

from langgraph.graph import END, StateGraph, START

### State
class GraphState(TypedDict):
    """
    Represents the state of graph of aspect agents.
    """
    
    original_query: str
    query: str
    aspect: str
    aspect_id: int
    answers_agent: Annotated[List[str], operator.add]
    ord_aspects: Annotated[List[str], operator.add]
    my_answer: str
    web_search: str
    documents: List[str]
    documents_kbt: List[str]
        
def rewrite_query(state,verbose,llm):
    if verbose: 
        print("---REWRITING QUERY---")
        print(f"State: {state}")
    original_query = state["original_query"]
    aspect = state["aspect"]
    
    prompt = get_prompt("/kaggle/input/prompts/prompts/query_rewriting.txt")
    examples = get_examples("/kaggle/input/prompts/prompts/query_rewriting_examples.txt")
    chain = call_model(llm, prompt, ["original_query","aspect"], examples)
    generation = chain.invoke({"original_query": original_query, "aspect": aspect})
    #print(list(generation.values())) #Debug
    return {"query": generation}


def retrieve(state,verbose,retriever,retrievers_KBT):
    if verbose: 
        print("---RETRIEVE---")
        print(f"State: {state}")
        
    query = state["query"]
    aspect_id = state["aspect_id"]

    # Retrieval
    documents = retriever.invoke(query)
    documents_kbt = retrievers_KBT[aspect_id].invoke(query)
    
    #pprint(f"Documents retrieved: {documents}")
    #pprint(f"Documents KBT retrieved: {documents_kbt}")
    
    return {"documents": documents, "documents_kbt": documents_kbt, "query": query}


def generate(state,verbose,llm,fairness):
    if verbose:
        print("---GENERATE---")
        print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]

    # RAG generation
    prompt = get_prompt("/kaggle/input/prompts/prompts/generating_answer.txt")
    chain = call_model(llm, prompt, ["context","question"])
    generation = chain.invoke({"context": documents, "question": query})
    
    aspect_id = state["aspect_id"]
    #print(f"Aspect agent {aspect_id} generates: {generation}") #Debug
    if fairness:
        return {"documents": documents, "query": query, "my_answer": generation}
    return {"documents": documents, "query": query, "my_answer": generation, "answers_agent": [generation], "ord_aspects": [state["aspect"]]}


def confirm_answer(state,verbose):
    if verbose:
        print("---CONFIRM ANSWER---")
        print(f"State: {state}")
    my_answer = state["my_answer"]

    return {"answers_agent": [my_answer], "ord_aspects": [state["aspect"]]}


def grade_documents(state,verbose,llm):
    if verbose:
        print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
        print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]
    
    prompt = get_prompt("/kaggle/input/prompts/prompts/retrieval_grader.txt")
    chain = call_model(llm, prompt, ["question","document"])

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = chain.invoke(
            {"question": query, "document": d.page_content}
        )
        #grade = score["score"]
        # Document relevant
        if "yes" in score.lower():
            if verbose: print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            if verbose: print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "query": query, "web_search": web_search}


def web_search(state,verbose):
    if verbose:
        print("---WEB SEARCH---")
        print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": query})
    #print(f"docs from web: {docs}") #Debug
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=250, chunk_overlap=0
    )

    doc_splits = text_splitter.split_documents([web_results])
    for doc in doc_splits:
        if documents is None:
            documents = [doc]
        else:
            documents.append(doc)
    return {"documents": documents, "query": query}


def hate_speech_filter(state,verbose,llm):
    if verbose:
        print("---HATE SPEECH FILTER---")
        print(f"State: {state}")
    documents = state["documents"]
    
    prompt = get_prompt("/kaggle/input/prompts/prompts/hate_speech.txt")
    examples = get_examples("/kaggle/input/prompts/prompts/hate_speech_examples.txt")
    chain = call_model(llm, prompt, ["input"], examples)

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = chain.invoke(
            {"input": d.page_content}
        )
        #grade = score["score"]
        if "no" in score.lower():
            if verbose: print("---DOCUMENT ACCEPTED---")
            filtered_docs.append(d)
    return {"documents": filtered_docs}


def entailment_filter(state,BART_model,strategy_entailment,neutral_acceptance,verbose,test_mode,observer,llm):    
    if verbose:
        print("---ENTAILMENT FILTER---")
        print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]
    documents_KBT = state["documents_kbt"]
    aspect_id = state["aspect_id"]
    
    prompt = get_prompt("/kaggle/input/prompts/prompts/entailment_checker.txt")
    examples = get_examples("/kaggle/input/prompts/prompts/entailment_checker_examples.txt")
    chain = call_model(llm, prompt, ["premise","hypothesis"], examples)

    # Score each doc
    filtered_docs = []
    #counter_docs = 0 # Debug
    if strategy_entailment: #Skeptical
        for d in documents:
            #counter_docs = counter_docs + 1 #Debug
            neutral = True
            for d_kbt in documents_KBT:
                if BART_model:
                    score = BART_prediction(d_kbt.page_content,d.page_content)
                else:
                    score = chain.invoke(
                        {"premise": d_kbt.page_content, "hypothesis": d.page_content}
                    )
                #grade = score["score"]
                
                #if score.lower() == "neutral": print(f"---Neutral {counter_docs}---") #Debug
                #if score.lower() == "entailment": print(f"---Entailment {counter_docs}---") #Debug
                #if score.lower() == "contradiction": print(f"---Contradiction {counter_docs}---") #Debug
                
                if not "neutral" in score.lower():
                    neutral = False
                if "contradiction" in score.lower():
                    # contradiction found
                    break
            if (not neutral or neutral_acceptance) and score.lower() != "contradiction":
                filtered_docs.append(d)
                #print(f"---Document accepted {counter_docs}---")  #Debug            
                if verbose: print("---DOCUMENT ENTAILED---")   
    else: #Credolous
        for d in documents:
            #counter_docs = counter_docs + 1 #Debug
            neutral = True
            for d_kbt in documents_KBT:
                if BART_model:
                    score = BART_prediction(d_kbt.page_content,d.page_content)
                else:
                    score = chain.invoke(
                        {"premise": d_kbt.page_content, "hypothesis": d.page_content}
                    )
                #grade = score["score"]
                
                #if score.lower() == "neutral": print(f"---Neutral {counter_docs}---") #Debug
                #if score.lower() == "entailment": print(f"---Entailment {counter_docs}---") #Debug
                #if score.lower() == "contradiction": print(f"---Contradiction {counter_docs}---") #Debug
                
                # Document entailed
                if not "neutral" in score.lower():
                    neutral = False
                if "entailment" in score.lower():
                    if verbose: print("---DOCUMENT ENTAILED---")
                    #print(f"---Document accepted {counter_docs}---")  #Debug
                    filtered_docs.append(d)
                    break
            if (neutral and neutral_acceptance) and score.lower() != "entailment":
                filtered_docs.append(d)
                #print(f"---Document accepted {counter_docs}---")  #Debug
                if verbose: print("---DOCUMENT ENTAILED---")
    
    if test_mode:
        #todo misura metriche con sklearn classifier
        #0 tweet veri, #1 tweet falsi
        y_true = [int(document.metadata.get("label")) for document in documents]
        y_pred = [0 if document in filtered_docs else 1 for document in documents]
        #print(f"Real docs with aspect {aspect_id} ,y_true: {y_true}") #Debug
        #print(f"Filtered docs with aspect {aspect_id} ,y_pred: {y_pred}") #Debug
        report = sklearn.metrics.classification_report(y_true,y_pred,labels=[0,1],
                                                       output_dict=True,zero_division=0)
        my_dict = {f"aspect_{aspect_id}": {"query": query, "report": report}}
        observer.generated_queries.update(my_dict)
        
    return {"documents": filtered_docs}


def debiasing(state,verbose,llm):
    if verbose:
        print("---DEBIASING FILTER---")
        print(f"State: {state}")
        
    answer = state["my_answer"]
    
    prompt = get_prompt("/kaggle/input/prompts/prompts/debiasing_answer.txt")
    examples = get_examples("/kaggle/input/prompts/prompts/debiasing_answer_examples.txt")
    chain = call_model(llm, prompt, ["text"], examples)
    
    unbiased_answer = chain.invoke({"text": answer})
    
    return {"answers_agent": [unbiased_answer], "ord_aspects": [state["aspect"]]}


### Conditional edge

def bias_detection(state,verbose,llm,bias_encoder_model):
    if verbose:
        print("---BIAS DETECTION---")
        print(f"State: {state}")
        
    answer = state["my_answer"]
    response = "biased"
    
    prompt = get_prompt("/kaggle/input/prompts/prompts/bias_detection.txt")
    examples = get_examples("/kaggle/input/prompts/prompts/bias_detection_examples.txt")
    chain = call_model(llm, prompt, ["input"], examples)
    
    if bias_encoder_model:
        bias_detection = pipeline('text-classification', model=bias_model, tokenizer=bias_model_tokenizer, device=device) # cuda = 0,1 based on gpu availability
        response = bias_detection(answer)[0]['label'].lower()
    else:
        response = chain.invoke({"input": answer})
        if "biased" in response: response = "biased"
        if "non-biased" in response: response = "non-biased"
        
    if verbose: print(response) #biased, non-biased
    
    return response


def decide_to_generate(state,verbose):
    if verbose:
        print("---ASSESS GRADED DOCUMENTS---")
        print(f"State: {state}")
    web_search = state["web_search"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        if verbose: print(
                "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---"
            )
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        if verbose: print("---DECISION: RELEVANT---")
        return "relevant"

**Building graph with edges**

In [16]:
# Workflow condizionale
def workflow_aspect_agent(configs):
    # Build graph
    workflow = StateGraph(GraphState)

    workflow.add_node("rewrite_query", functools.partial(rewrite_query, verbose=configs.verbose, 
                                                    llm=configs.local_llm))  # query rewriting
    workflow.add_node("retrieve", functools.partial(retrieve, verbose=configs.verbose, 
                                                    retriever=configs.retriever, retrievers_KBT=configs.retrievers_KBT))  # retrieve
    workflow.add_node("generate", functools.partial(generate, verbose=configs.verbose, 
                                                    llm=configs.local_llm, fairness=configs.fairness))  # generatae
    
    if configs.web_search:
        workflow.add_node("websearch", functools.partial(web_search, verbose=configs.verbose))  # web search
        workflow.add_node("grade_documents", functools.partial(grade_documents, verbose=configs.verbose, 
                                                               llm=configs.local_llm))  # grade documents
    if configs.safeness:
        workflow.add_node("hate_speech_filter", functools.partial(hate_speech_filter, verbose=configs.verbose, 
                                                                  llm=configs.local_llm))
    if configs.trustworthiness:
        workflow.add_node("entailment_filter", functools.partial(entailment_filter, BART_model=configs.BART_model, strategy_entailment=configs.strategy_entailment, 
                                                                 neutral_acceptance=configs.neutral_acceptance, verbose=configs.verbose, llm=configs.local_llm, 
                                                                 test_mode=configs.test_mode, observer=configs.observer))  # entailment
    if configs.fairness:  
        workflow.add_node("debiasing_filter", functools.partial(debiasing, verbose=configs.verbose, llm=configs.local_llm))
        workflow.add_node("confirm_answer", functools.partial(confirm_answer, verbose=configs.verbose))

    # Non applichiamo il routing
    """
    workflow.add_conditional_edges(
        START,
        route_question,
        {
            "websearch": "websearch", #se la risposta è websearch, allora vai al nodo websearch
            "vectorstore": "retrieve", #se la risposta è vectorstore, allora vai al nodo retrieve
        },
    )
    """
    workflow.add_edge(START, "rewrite_query")
    workflow.add_edge("rewrite_query", "retrieve")
    
    if configs.web_search:
        workflow.add_edge("retrieve", "grade_documents")  
        workflow.add_conditional_edges(
            "grade_documents",
            functools.partial(decide_to_generate, verbose=configs.verbose),
            {
                "websearch": "websearch",
                "relevant": "hate_speech_filter" if configs.safeness else "entailment_filter" if configs.trustworthiness else "generate"
            },
        )
    
    if configs.safeness:
        if configs.web_search:
            workflow.add_edge("websearch", "hate_speech_filter")
        else: 
            workflow.add_edge("retrieve", "hate_speech_filter")
        if configs.trustworthiness:
            workflow.add_edge("hate_speech_filter", "entailment_filter")
            workflow.add_edge("entailment_filter", "generate")
        else:
            workflow.add_edge("hate_speech_filter","generate")
    elif configs.trustworthiness:
        if configs.web_search:
            workflow.add_edge("websearch", "entailment_filter")
        else: 
            workflow.add_edge("retrieve", "entailment_filter")
        workflow.add_edge("entailment_filter", "generate")
    else:
        if configs.web_search:
            workflow.add_edge("websearch", "generate")
        else: 
            workflow.add_edge("retrieve", "generate")
    
    if configs.fairness:
        workflow.add_conditional_edges(
            "generate",
            functools.partial(bias_detection, verbose=configs.verbose, llm=configs.local_llm, bias_encoder_model=configs.bias_encoder_model),
            {
                "biased": "debiasing_filter",
                "non-biased": "confirm_answer",
            },
        )
        workflow.add_edge("confirm_answer", END)
        workflow.add_edge("debiasing_filter", END)
    else:
        workflow.add_edge("generate", END)

    # Non faccio il controllo sulle allucinazioni
    """
    workflow.add_conditional_edges(
        "generate",
        grade_generation_v_documents_and_question,
        {
            "not supported": "generate",
            "useful": END,
            "not useful": "websearch",
        },
    )
    """    
    workflow_compiled = workflow.compile()
    return workflow_compiled

In [ ]:
from IPython.display import Image, display

display(Image(workflow_aspect_agent(configs).get_graph().draw_mermaid_png()))

# Master agent

In [17]:
from typing import Annotated
import operator
from langgraph.constants import Send


### Super Graph State
class SuperGraphState(TypedDict):
    """
    Represents the state of our super-graph.
    """
    
    question: str
    aspects: List[str]
    answers_agent: Annotated[List[str], operator.add]
    ord_aspects: Annotated[List[str], operator.add]
    final_answer: str

def send_aspects(state,verbose):
    if verbose: 
        print("---SEND ASPECT TO EACH ASPECT-AGENT---")
        print(f"State: {state}")
    return [Send("aspect_agent_node", {"original_query": state["question"], "aspect": a, "aspect_id": state["aspects"].index(a)}) for a in state["aspects"]]

def organize_answers(state,verbose,llm,organize):
    if verbose:
        print("---ORGANIZE OUTPUTS---")
        print(f"State: {state}")
    answers_agent = state["answers_agent"]
    ord_aspects = state["ord_aspects"]
    
    #print(f"I'm the master agent and I received: {answers_agent}") #Debug
    if organize: #with sections
        prompt = get_prompt("/kaggle/input/prompts/prompts/final_answer_with_sections.txt")
        chain = call_model(llm, prompt, ["answers","aspects"])
        final_output=chain.invoke({"answers": answers_agent, "aspects": ord_aspects})
    else: #without sections
        prompt = get_prompt("/kaggle/input/prompts/prompts/final_answer.txt")
        chain = call_model(llm, prompt, ["answers"])
        final_output=chain.invoke({"answers": answers_agent})
    return {"final_answer": final_output}

In [18]:
def master_flow(configs):
    master_flow = StateGraph(SuperGraphState)

    # Define the nodes
    master_flow.add_node("organize_answers", functools.partial(organize_answers, verbose=configs.verbose, llm=configs.local_llm, organize=configs.organize))
    master_flow.add_node("aspect_agent_node",workflow_aspect_agent(configs))

    # Build graph
    master_flow.add_conditional_edges(START, functools.partial(send_aspects, verbose=configs.verbose), ["aspect_agent_node"])
    master_flow.add_edge("aspect_agent_node", "organize_answers")
    master_flow.add_edge("organize_answers", END)

    master_compiled = master_flow.compile()
    return master_compiled

In [ ]:
from IPython.display import Image, display

# Setting xray to 1 will show the internal structure of the nested graph
display(Image(master_flow(configs).get_graph().draw_mermaid_png()))

# Configuration and app-launching

Vectorstore configuration

In [21]:
index_name = "entailment-test"
aspects = ["Health","Technology","Society"] #"Technology", "Society"

top_retriever = 20 #documents retrieved by retriever
top_KBT = 5 #documents retrievede by KBT retriever

retriever = create_retriever(index_name, top_retriever)
retrievers_KBT = create_KBT_retrievers(aspects, top_KBT)

In [22]:
class Config(object):
    def __init__(self,retriever,retrievers_KBT,aspects):
        self.local_llm = "llama3.1" #"gemma2"
        
        # retrievers
        self.retriever = retriever
        self.retrievers_KBT = retrievers_KBT
        self.aspects = aspects
        
        #if we want print all the process: True
        self.verbose = False 
        
        #if we want to include websearch in the workflow
        self.web_search = False

        # Controlling properties
        self.safeness = False # if we want to add hate speech detection module
        self.trustworthiness = True # if we want to add entailment module with KBT
        self.fairness = False  # if we want to add debiasing module.

        #Controlling entailment
        # strategy for the entailment, False = "Credolous", True = "Skeptical" 
        self.strategy_entailment = True
        #manage the total neutral entailed documents (what if a document is neutral with all documents of KBT)
        # True = accept the neutral documents, False = don't accept
        self.neutral_acceptance = True   
        # True: uses BART model for the entailment, False: uses LLM
        self.BART_model = False
        
        #controlling bias detection
        # True: uses encoder model. False: uses LLM
        self.bias_encoder_model = False
        
        #controlling final generation 
        # True: organize final output in section. False: organize output without sections
        self.organize = False
        
        # For testing
        self.test_mode = True
        self.observer = None
    
configs = Config(retriever,retrievers_KBT,aspects)

Starting language model

In [25]:
start_ollama()
pull_model(configs.local_llm)
start_model(configs.local_llm)

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
2024/08/19 07:37:59 routes.go:1125: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIJG/c/XHYXFQxkiC54YG+cyh8vJ08FKcgUy0qlziAibv



time=2024-08-19T07:38:06.807Z level=INFO source=payload.go:44 msg="Dynamic LLM libraries [cpu cpu_avx cpu_avx2 cuda_v11 rocm_v60102]"
time=2024-08-19T07:38:06.807Z level=INFO source=gpu.go:204 msg="looking for compatible GPUs"


[GIN] 2024/08/19 - 07:38:07 | 200 |      56.412µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ 

time=2024-08-19T07:38:07.032Z level=INFO source=types.go:105 msg="inference compute" id=GPU-63131bf6-0d47-0b93-b00c-19be01a23bc6 library=cuda compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="936.1 MiB"
time=2024-08-19T07:38:07.033Z level=INFO source=types.go:105 msg="inference compute" id=GPU-4c6c6250-c736-f0bb-1dd3-f2fe7aa62bf8 library=cuda compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.5 GiB"


...

[GIN] 2024/08/19 - 07:38:53 | 200 | 46.089930343s |       127.0.0.1 | POST     "/api/pull"
[GIN] 2024/08/19 - 07:38:53 | 200 |   23.289929ms |       127.0.0.1 | POST     "/api/show"


pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 11ce4ee3e170... 100% ▕████████████████▏ 1.7 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1a4c3c319823... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 
⠙ ⠹ time=2024-08-19T07:38:53.982Z level=INFO source=sched.go:710 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-8eeb52dfb3bb9aefdf9d1ef24b3bdbcfbe82238798c4b918278320b6fcef18fe gpu=GPU-4c6c6250-c736-f0bb-1dd3-f2fe7aa62bf8 parallel=4 available=15615524864 required="6.2 GiB"
time=2024-08-19T07:38:53.982Z level=INFO source=memory.go:309 msg="offload to cuda" layers.requested=-1 layers.model=33 layers.offload=33 layers.split="" memory.availabl

INFO [main] build info | build=1 commit="1e6f655" tid="134026648657920" timestamp=1724053133
INFO [main] system info | n_threads=2 n_threads_batch=-1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="134026648657920" timestamp=1724053133 total_threads=4
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="36149" tid="134026648657920" timestamp=1724053133


llama_model_loader: - kv  23:                  tokenizer.ggml.token_type arr[i32,128256]  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
time=2024-08-19T07:38:54.236Z level=INFO source=server.go:627 msg="waiting for server to become available" status="llm server loading model"
⠴ llama_model_loader: - kv  24:                      tokenizer.ggml.merges arr[str,280147]  = ["Ġ Ġ", "Ġ ĠĠĠ", "ĠĠ ĠĠ", "...
llama_model_loader: - kv  25:                tokenizer.ggml.bos_token_id u32              = 128000
llama_model_loader: - kv  26:                tokenizer.ggml.eos_token_id u32              = 128009
llama_model_loader: - kv  27:                    tokenizer.chat_template str              = {{- bos_token }}\n{%- if custom_tools ...
llama_model_loader: - kv  28:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   66 tensors
llama_model_loader: - type q4_0:  225 tensors
llama_model_loader: - type q6_K:    1 tensors
⠦ ⠧ ⠇ ⠏ ⠋ llm_load_vocab: special tok

INFO [main] model loaded | tid="134026648657920" timestamp=1724053137
[GIN] 2024/08/19 - 07:38:57 | 200 |  3.964513225s |       127.0.0.1 | POST     "/api/generate"


In [ ]:
inputs = {"question": "Covid19 was a hoax?", "aspects": configs.aspects}

for output in master_flow(configs).stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
answer = value["final_answer"]
print(answer)

# Test

In [23]:
# Funzioni di supporto:
import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)

"""
queries_gpt = ["Does the COVID-19 vaccine contain a microchip for controlling people?",
"Was the COVID-19 pandemic planned by a global elite?",
"Does inhaling hot steam kill the COVID-19 virus?",
"Was COVID-19 created in a lab as a biological weapon?",
"Does drinking alcohol help prevent COVID-19 infection?",
"Is 5G responsible for spreading COVID-19?",
"Do masks cause a reduction in blood oxygen levels?",
"Do COVID-19 vaccines cause infertility in women?",
"Can high doses of vitamin C cure COVID-19?",
"Can pets transmit COVID-19 to humans?"]
"""

queries_gpt = ["Tell me about the Covid-19 pandemic.",
"What measures were taken to manage the Covid-19 emergency?",
"What were the repercussions of the Covid-19 pandemic?",
"How did people experience the Covid-19 pandemic?",
"Tell me about the misinformation spread during the Covid-19 pandemic."]

#write_file("/kaggle/working/gpt_queries.json",queries_gpt)

Predisposizione dell'observer per salvare i risultati

In [24]:
class Observer(object):
    def __init__(self):
        self.query=""
        self.type_of_acceptance=""
        self.neutral_acceptance=False
        self.generated_queries={}
        self.final_answer=""
    
    def generate_dict(self):
        return {"query": self.query,
               "type_of_acceptance": self.type_of_acceptance,
               "neutral_acceptance": self.neutral_acceptance,
               "generated_queries": self.generated_queries,
               "final_answer": self.final_answer}

Combinazioni di configurazione con skeptical/credulous e neutral

Multiple combination:

In [26]:
combination = [(True,True), (True,False), (False,True),(False,False)] #Skeptical-Neutral #Skeptical-No-Neutral #Cred-Neu #Cred-No-Neu

for comb in combination:
    print("Start combination")
    configs.strategy_entailment =  comb[0]
    configs.neutral_acceptance = comb[1]
    
    queries_list = queries_gpt

    attempt = 1
    ret_dict = {}
    for query in queries_list:
        inputs = {"question": query, "aspects": configs.aspects}
        print(f"Attempt {attempt} start")
    
        configs.observer = Observer()
        configs.observer.query=query
        if configs.strategy_entailment:
            configs.observer.type_of_acceptance="Skeptical"
        else:
            configs.observer.type_of_acceptance="Credulous"
        configs.observer.neutral_acceptance=configs.neutral_acceptance

        for output in master_flow(configs).stream(inputs):
            for key, value in output.items():
                pass
                #pprint(f"Finished running: {key}:")
        answer = value["final_answer"]
    
        configs.observer.final_answer= answer
        ret_dict[f"attempt {attempt}"] = configs.observer.generate_dict()
    
        attempt = attempt + 1

    stringa = "Neutral" if configs.neutral_acceptance else "No-Neutral"
    write_file(f"/kaggle/working/test_llama31_newq_{configs.observer.type_of_acceptance}_{stringa}.json",ret_dict)

Start combination
Attempt 1 start
[GIN] 2024/08/19 - 07:41:15 | 200 |  983.086095ms |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/19 - 07:41:15 | 200 |  1.220373756s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/19 - 07:41:15 | 200 |  1.273680533s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/19 - 07:41:19 | 200 |  1.948057186s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/19 - 07:41:19 | 200 |   2.17546901s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/19 - 07:41:20 | 200 |   2.43238964s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/19 - 07:41:20 | 200 |  879.447958ms |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/19 - 07:41:20 | 200 |  798.743614ms |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/19 - 07:41:20 | 200 |  745.032971ms |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/19 - 07:41:21 | 200 |  753.507888ms |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/19

Single combination:

In [ ]:
#queries_list = load_queries("/kaggle/input/preference/gpt_queries.json")
queries_list = queries_gpt

attempt = 1
ret_dict = {}
for query in queries_list:
    inputs = {"question": query, "aspects": configs.aspects}
    print(f"Attempt {attempt} start")
    
    configs.observer = Observer()
    configs.observer.query=query
    if configs.strategy_entailment:
        configs.observer.type_of_acceptance="Skeptical"
    else:
        configs.observer.type_of_acceptance="Credulous"
    configs.observer.neutral_acceptance=configs.neutral_acceptance

    for output in master_flow(configs).stream(inputs):
        for key, value in output.items():
            pass
            #pprint(f"Finished running: {key}:")
    answer = value["final_answer"]
    
    configs.observer.final_answer= answer
    ret_dict[f"attempt {attempt}"] = configs.observer.generate_dict()
    
    attempt = attempt + 1

stringa = "Neutral" if configs.neutral_acceptance else "No-Neutral"
write_file(f"/kaggle/working/test_llama31_{configs.observer.type_of_acceptance}_{stringa}.json",ret_dict)

In [ ]:
print(ret_dict)

# Evaluation

## Entailment evaluation

In [7]:
import pandas as pd

### Valutazione su notizie vere (0) e notizie false (1)

In [8]:
# Label 0, notizie vere
# Label 1, notizie false

def results_on_label(label, num_attempts, num_aspects, dict_input):
    recall = 0
    precision = 0
    f1 = 0
    support = 0

    for i in range(1,num_attempts+1):
        for j in range(num_aspects):
            recall = recall + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{j}"]["report"][label]["recall"]
            precision = precision + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{j}"]["report"][label]["precision"]
            f1 = f1 + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{j}"]["report"][label]["f1-score"]
            support = support + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{j}"]["report"][label]["support"]

    return {"recall":recall/(num_attempts*num_aspects), 
            "precision":precision/(num_attempts*num_aspects),
            "f1":f1/(num_attempts*num_aspects), 
            "support":support/(num_attempts*num_aspects)}

### Valutazione per aspetto

In [9]:
def results_on_aspects(aspect_id, num_attempts, num_aspects, dict_input):
    recall = 0
    precision = 0
    f1 = 0

    weight_recall = 0
    weight_precision = 0
    weight_f1 = 0
    
    macro_recall = 0
    macro_precision = 0
    macro_f1 = 0
    
    accuracy = 0

    for i in range(1,num_attempts+1):
        recall = recall + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["macro avg"]["recall"]
        precision = precision + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["macro avg"]["precision"]
        f1 = f1 + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["macro avg"]["f1-score"]
        
        macro_recall = macro_recall + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["macro avg"]["recall"]
        macro_precision = macro_precision + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["macro avg"]["precision"]
        macro_f1 = macro_f1 + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["macro avg"]["f1-score"]
        
        weight_recall = weight_recall + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["weighted avg"]["recall"]
        weight_precision = weight_precision + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["weighted avg"]["precision"]
        weight_f1 = weight_f1 + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["weighted avg"]["f1-score"]
        try:
            accuracy = accuracy + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["accuracy"]
        except:
            accuracy = accuracy + 1 #se l'accuracy non è presente nel dizionario, è perché vale 1 (vedi esempio 4 di llama_second_skeptical_neutral)
    
    return {"recall": recall/(num_attempts),
            "precision": precision/(num_attempts), 
            "f1": f1/(num_attempts),
            "macro_recall": macro_recall/(num_attempts),
            "macro_precision": macro_precision/(num_attempts),
            "macro_f1": macro_f1/(num_attempts),
            "w_recall": weight_recall/(num_attempts),
            "w_precision": weight_precision/(num_attempts), 
            "w_f1":weight_f1/(num_attempts), 
            "accuracy":accuracy/(num_attempts)}

In [10]:
import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)

In [27]:
# label 0: notizie vere (negative class)
# label 1: notizie false (positive class)
import pandas as pd

def compute_result_on_label(model, shot, acceptance, neutral, num_queries, num_aspects) -> pd.DataFrame:
    if shot=="first" or model=="BART" or model=="gemma2":
        dict_input = load_json(f"/kaggle/working/test_{model}_{acceptance}_{neutral}.json")
    else:
        #dict_input = load_json(f"/kaggle/working/test_{model}_{shot}_{acceptance}_{neutral}.json")
        dict_input = load_json(f"/kaggle/working/test_{model}_newq_{acceptance}_{neutral}.json")

    results_label_0 = results_on_label("0", num_queries, num_aspects, dict_input)
    results_label_1 = results_on_label("1", num_queries, num_aspects, dict_input)
    

    df = pd.DataFrame()
    df["model"] = [model]
    df["shots"] = [shot]
    df["num_queries"] = [num_queries]
    df["retrieved"] = [10 if num_queries==10 else 20]
    df["type_acceptance"] = [acceptance]
    df["neutral"] = [neutral]

    df["precision_0"] = results_label_0["precision"]
    df["recall_0"] = results_label_0["recall"]
    df["f1_0"] = results_label_0["f1"]
    df["support_0"] = results_label_0["support"]
    

    df["precision_1"] = results_label_1["precision"]
    df["recall_1"] = results_label_1["recall"]
    df["f1_1"] = results_label_1["f1"]
    df["support_1"] = results_label_1["support"]

    return df

In [28]:
def compute_result_on_aspects(model, shot, acceptance, neutral, num_queries, num_aspects, aspects) -> pd.DataFrame:
    if shot=="first" or model=="BART" or model=="gemma2":
        dict_input = load_json(f"/kaggle/working/test_{model}_{acceptance}_{neutral}.json")
    else:
        #dict_input = load_json(f"/kaggle/working/test_{model}_{shot}_{acceptance}_{neutral}.json")
        dict_input = load_json(f"/kaggle/working/test_{model}_newq_{acceptance}_{neutral}.json")

    df = pd.DataFrame()
    df["model"] = [model]
    df["shots"] = [shot]
    df["num_queries"] = [num_queries]
    df["retrieved"] = [10 if num_queries==10 else 20]
    df["type_acceptance"] = [acceptance]
    df["neutral"] = [neutral]
    
    aspect_id = 0
    for aspect in aspects:
        result = results_on_aspects(aspect_id, num_queries, num_aspects, dict_input)
        df[f"precision_{aspect}"] = result["precision"]
        df[f"recall_{aspect}"] = result["recall"]
        df[f"f1_{aspect}"] = result["f1"]
        df[f"accuracy_{aspect}"] = result["accuracy"]
        
        df[f"macro_precision_{aspect}"] = result["macro_precision"]
        df[f"macro_recall_{aspect}"] = result["macro_recall"]
        df[f"macro_f1_{aspect}"] = result["macro_f1"]
        
        df[f"w_precision_{aspect}"] = result["w_precision"]
        df[f"w_recall_{aspect}"] = result["w_recall"]
        df[f"w_f1_{aspect}"] = result["w_f1"]
        
        aspect_id = aspect_id+1    

    return df

In [21]:
import openpyxl

attempts = 10
num_aspects = 3

models = ["BART","llama31"] #["gemma2"]
shots = ["first","second","third"]
type_acceptance = ["Skeptical","Credulous"]
neutral_acceptance = ["No-Neutral","Neutral"]
aspects = ["Health","Technology","Society"]

excel_file = "/kaggle/working/test_entailment_labels_new.xlsx"
results = pd.DataFrame()
for model in models:
    for shot in shots:
        for acceptance in type_acceptance:
            for neutral in neutral_acceptance:
                new_row = compute_result_on_label(model, shot, acceptance, neutral, attempts, num_aspects)
                results = pd.concat([results, new_row], ignore_index=True)
        if model=="BART": break

print(results.round(3))
#results.round(3).to_excel(excel_file, index=False, engine='openpyxl')

      model   shots  num_queries  retrieved type_acceptance     neutral  \
0      BART   first           10         10       Skeptical  No-Neutral   
1      BART   first           10         10       Skeptical     Neutral   
2      BART   first           10         10       Credulous  No-Neutral   
3      BART   first           10         10       Credulous     Neutral   
4   llama31   first           10         10       Skeptical  No-Neutral   
5   llama31   first           10         10       Skeptical     Neutral   
6   llama31   first           10         10       Credulous  No-Neutral   
7   llama31   first           10         10       Credulous     Neutral   
8   llama31  second           10         10       Skeptical  No-Neutral   
9   llama31  second           10         10       Skeptical     Neutral   
10  llama31  second           10         10       Credulous  No-Neutral   
11  llama31  second           10         10       Credulous     Neutral   
12  llama31   third      

In [30]:
attempts = 5
num_aspects = 3

models = ["llama31"]#["BART","llama31"] #["gemma2"]
shots = [12] #["first","second","third"]
type_acceptance = ["Skeptical","Credulous"]
neutral_acceptance = ["No-Neutral","Neutral"]
aspects = ["Health","Technology","Society"]

excel_file = "/kaggle/working/test_entailment_aspects_new.xlsx"
#results = pd.DataFrame()
for model in models:
    for shot in shots:
        for acceptance in type_acceptance:
            for neutral in neutral_acceptance:
                new_row = compute_result_on_aspects(model, shot, acceptance, neutral, attempts, num_aspects, aspects)
                results = pd.concat([results, new_row], ignore_index=True)
        if model=="BART": break

print(results.round(3))
results.round(3).to_excel(excel_file, index=False, engine='openpyxl')

      model   shots  num_queries  retrieved type_acceptance     neutral  \
0      BART   first           10         10       Skeptical  No-Neutral   
1      BART   first           10         10       Skeptical     Neutral   
2      BART   first           10         10       Credulous  No-Neutral   
3      BART   first           10         10       Credulous     Neutral   
4   llama31   first           10         10       Skeptical  No-Neutral   
5   llama31   first           10         10       Skeptical     Neutral   
6   llama31   first           10         10       Credulous  No-Neutral   
7   llama31   first           10         10       Credulous     Neutral   
8   llama31  second           10         10       Skeptical  No-Neutral   
9   llama31  second           10         10       Skeptical     Neutral   
10  llama31  second           10         10       Credulous  No-Neutral   
11  llama31  second           10         10       Credulous     Neutral   
12  llama31   third      

# Ragas (application evaluation)

https://docs.ragas.io/en/stable/

In [ ]:
#todo